In [16]:
# config.py
import torch

class Config:
    seed = 42

    # 데이터 경로
    train_path = "../database/train/train_augmented.csv"
    test_path  = "../database/test/test.csv"

    # 학습 파라미터
    batch_size = 8
    num_epochs = 20       # 메인 모델 학습 에폭
    lr = 3e-4

    # CBOW 학습 파라미터
    cbow_num_epochs = 20   # CBOW 모델 학습 에폭 (예시)
    cbow_lr = 1e-3
    window_size = 3       # CBOW의 윈도우 사이즈
    batch_size_cbow = 4096

    # 모델 파라미터
    embed_dim = 512
    hidden_size = 512
    num_layers = 4  # GRU 레이어 수

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 특수 토큰
    PAD_TOKEN = "[PAD]"
    UNK_TOKEN = "[UNK]"


In [17]:
# dataset.py
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
import numpy as np

# 기존 CharTokenizer와 TokenClassifyDataset
class CharTokenizer:
    def __init__(self, vocab, pad_token, unk_token):
        self.vocab = vocab
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.pad_id = vocab[pad_token]
        self.unk_id = vocab[unk_token]
        self.id2token = {v: k for k, v in vocab.items()}

    def encode(self, text):
        return [self.vocab.get(ch, self.unk_id) for ch in text]

    def decode(self, ids):
        return "".join([self.id2token.get(i, "") for i in ids if i in self.id2token])


class TokenClassifyDataset(Dataset):
    """
    난독화 해제용 (입출력 길이 무관) Dataset
    (input_str, output_str)을 (input_ids, label_ids)로 변환
    """
    def __init__(self, pairs, tokenizer):
        """
        pairs: list of (input_str, output_str)
        tokenizer: CharTokenizer
        """
        self.samples = []
        self.tokenizer = tokenizer

        for inp, outp in pairs:
            # 1) 전처리: 문장 끝의 공백 제거
            inp = inp.strip()
            outp = outp.strip()

            # 2) 토큰화
            input_ids = tokenizer.encode(inp)
            label_ids = tokenizer.encode(outp)

            # 3) 길이가 달라도 스킵하지 않고 그대로 저장
            self.samples.append((input_ids, label_ids))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


def build_vocab(pairs, special_tokens):
    """
    pairs: list of (input_str, output_str)
    special_tokens: [PAD, UNK] 등
    """
    chars = set()
    for inp, outp in pairs:
        chars.update(list(inp))
        chars.update(list(outp))

    vocab = {}
    for sp in special_tokens:
        vocab[sp] = len(vocab)
    for c in sorted(list(chars)):
        if c not in vocab:
            vocab[c] = len(vocab)
    return vocab


def token_collate_fn(batch):
    """
    batch: list of (input_ids, label_ids)
    """
    import torch
    input_list, label_list = [], []
    for inp, lab in batch:
        input_list.append(torch.tensor(inp, dtype=torch.long))
        label_list.append(torch.tensor(lab, dtype=torch.long))

    # pad_sequence => (B, T)
    input_padded = nn.utils.rnn.pad_sequence(
        input_list, batch_first=True, padding_value=0
    )
    label_padded = nn.utils.rnn.pad_sequence(
        label_list, batch_first=True, padding_value=0
    )
    return input_padded, label_padded


# ===== CBOW 데이터셋 및 collate 함수 추가 =====
class CBOWDataset(Dataset):
    """
    CBOW 학습용 데이터셋: 각 텍스트에서 각 토큰을 대상으로, 주변 window 내 단어(문자)를 context로,
    해당 단어를 target으로 생성.
    """
    def __init__(self, texts, tokenizer, window_size):
        self.samples = []
        self.tokenizer = tokenizer
        for text in texts:
            text = text.strip()
            tokens = tokenizer.encode(text)
            for i in range(len(tokens)):
                start = max(0, i - window_size)
                end = min(len(tokens), i + window_size + 1)
                # i번째 단어 제외한 context
                context = tokens[start:i] + tokens[i+1:end]
                if len(context) > 0:
                    self.samples.append((context, tokens[i]))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]


def cbow_collate_fn(batch):
    """
    batch: list of (context (list of token ids), target (token id))
    """
    contexts, targets = zip(*batch)
    context_tensors = [torch.tensor(ctx, dtype=torch.long) for ctx in contexts]
    contexts_padded = nn.utils.rnn.pad_sequence(
        context_tensors, batch_first=True, padding_value=0
    )
    targets_tensor = torch.tensor(targets, dtype=torch.long)
    return contexts_padded, targets_tensor


In [18]:
# model.py
import torch
import torch.nn as nn

# 기존 BiMultiGRUModel
class BiMultiGRUModel(nn.Module):
    """
    양방향, 다층 GRU 기반 모델
    """
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.gru = nn.GRU(
            input_size=embed_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.output_layer = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, input_ids):
        """
        input_ids: (B, T)
        return: (B, T, vocab_size)
        """
        x = self.embedding(input_ids)      # (B, T, E)
        outputs, _ = self.gru(x)             # (B, T, 2H)
        logits = self.output_layer(outputs)  # (B, T, vocab_size)
        return logits


# ===== CBOW 모델 정의 =====
class CBOW(nn.Module):
    def __init__(self, vocab_size, embed_dim, pad_idx=0):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.linear = nn.Linear(embed_dim, vocab_size)
    
    def forward(self, contexts):
        """
        contexts: (B, T) - 패딩된 context 단어 id 시퀀스
        """
        # 임베딩 얻기
        emb = self.embedding(contexts)  # (B, T, embed_dim)
        # 패딩 마스크 (pad_idx==0)
        mask = (contexts != 0).unsqueeze(-1).float()  # (B, T, 1)
        emb_masked = emb * mask  # (B, T, embed_dim)
        # 각 샘플의 실제 길이 (패딩 제외)
        lengths = mask.sum(dim=1)  # (B, 1)
        # 평균 임베딩 (패딩 토큰은 0으로 처리)
        avg_emb = emb_masked.sum(dim=1) / lengths.clamp(min=1)
        logits = self.linear(avg_emb)  # (B, vocab_size)
        return logits

# class BiMultiLSTMModel(nn.Module):
#     """
#     양방향, 다층 LSTM 기반 모델
#     """
#     def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, pad_idx=0):
#         super().__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

#         # 양방향 LSTM 적용
#         self.lstm = nn.LSTM(
#             input_size=embed_dim,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=True
#         )

#         # 양방향이므로 hidden_size * 2
#         self.fc = nn.Linear(hidden_size * 2, vocab_size)

#     def forward(self, x):
#         x = self.embedding(x)  # (B, T, embed_dim)
#         output, _ = self.lstm(x)  # (B, T, hidden_size * 2)
#         logits = self.fc(output)  # (B, T, vocab_size)
#         return logits

# class BiMultiLSTMModel(nn.Module):
#     """
#     양방향, 다층 LSTM 기반 모델 (Dropout 적용)
#     """
#     def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, dropout=0.5, pad_idx=0):
#         super().__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
#         self.dropout = nn.Dropout(dropout)  # Embedding 이후 Dropout 추가

#         # 양방향 LSTM 적용
#         self.lstm = nn.LSTM(
#             input_size=embed_dim,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=True,
#             dropout=dropout if num_layers > 1 else 0  # 단층 LSTM일 경우 dropout 적용 안 함
#         )

#         # 양방향이므로 hidden_size * 2
#         self.fc = nn.Linear(hidden_size * 2, vocab_size)
#         self.out_dropout = nn.Dropout(dropout)  # 최종 출력에 Dropout 추가

#     def forward(self, x):
#         x = self.embedding(x)  # (B, T, embed_dim)
#         x = self.dropout(x)  # Embedding Dropout
#         output, _ = self.lstm(x)  # (B, T, hidden_size * 2)
#         output = self.out_dropout(output)  # LSTM 이후 Dropout 추가
#         logits = self.fc(output)  # (B, T, vocab_size)
#         return logits
    

class BiMultiLSTMModel(nn.Module):
    """
    4층 양방향 LSTM + LayerNorm 적용 + Dropout 포함
    """
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, dropout=0.5, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.embed_dropout = nn.Dropout(dropout)

        # LayerNorm 추가
        self.layer_norm_input = nn.LayerNorm(embed_dim)  # LSTM 입력 전 정규화
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # LSTM hidden state 정규화
        self.layer_norm_lstm = nn.LayerNorm(hidden_size * 2)  # BiLSTM이므로 hidden_size * 2
        
        self.fc = nn.Linear(hidden_size * 2, vocab_size)
        self.out_dropout = nn.Dropout(dropout)  # 최종 출력 dropout

    def forward(self, x):
        x = self.embedding(x)  # (B, T, embed_dim)
        x = self.embed_dropout(x)  # Embedding Dropout
        
        x = self.layer_norm_input(x)  # LSTM 입력 전에 정규화
        output, _ = self.lstm(x)  # (B, T, hidden_size * 2)

        output = self.layer_norm_lstm(output)  # LSTM hidden state 정규화
        output = self.out_dropout(output)  # LSTM 이후 Dropout 추가
        logits = self.fc(output)  # (B, T, vocab_size)
        return logits





In [19]:
# train.py
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd



def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def train_model(model, dataloader, config):
    model.to(config.device)
    optimizer = optim.Adam(model.parameters(), lr=config.lr)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # pad=0

    for epoch in range(1, config.num_epochs + 1):
        model.train()
        total_loss = 0
        for batch in dataloader:
            input_ids, label_ids = batch
            input_ids = input_ids.to(config.device)
            label_ids = label_ids.to(config.device)

            logits = model(input_ids)  # (B, T, vocab_size)
            B, T, V = logits.size()

            loss = criterion(
                logits.view(-1, V),
                label_ids.view(-1)
            )
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"[Epoch {epoch}/{config.num_epochs}] Loss: {avg_loss:.4f}")

# # 다층 lstm 학습코드드
# def train_model(model, dataloader, config):
#     model.to(config.device)
#     optimizer = optim.Adam(model.parameters(), lr=config.lr)
#     criterion = nn.CrossEntropyLoss(ignore_index=0)  # pad=0

#     for epoch in range(1, config.num_epochs + 1):
#         model.train()
#         total_loss = 0
#         for batch in dataloader:
#             input_ids, label_ids = batch
#             input_ids = input_ids.to(config.device)
#             label_ids = label_ids.to(config.device)

#             logits = model(input_ids)  # (B, T, vocab_size)
#             B, T, V = logits.size()

#             loss = criterion(
#                 logits.view(-1, V),
#                 label_ids.view(-1)
#             )
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()

#         avg_loss = total_loss / len(dataloader)
#         print(f"[Epoch {epoch}/{config.num_epochs}] Loss: {avg_loss:.4f}")


def predict_model(model, dataloader, tokenizer, config):
    model.eval()
    results = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, label_ids = batch
            input_ids = input_ids.to(config.device)
            
            logits = model(input_ids)  # (B, T, vocab_size)
            preds = logits.argmax(dim=-1)  # (B, T)

            # CPU로 복사
            preds_cpu = preds.cpu().numpy().tolist()
            label_cpu = label_ids.numpy().tolist()
            input_cpu = input_ids.cpu().numpy().tolist()

            # 각 샘플별 처리
            for inp_ids, lab_ids, pred_ids in zip(input_cpu, label_cpu, preds_cpu):
                real_len = sum([1 for x in inp_ids if x != 0])  # 패딩 제외 길이
                pred_ids_sliced = pred_ids[:real_len]
                inp_str  = tokenizer.decode(inp_ids[:real_len])
                lab_str  = tokenizer.decode(lab_ids[:real_len])
                pred_str = tokenizer.decode(pred_ids_sliced)
                results.append((inp_str, lab_str, pred_str))
    return results


# ===== CBOW 모델 학습 함수 =====
def train_cbow_model(model, dataloader, config):
    model.to(config.device)
    optimizer = optim.Adam(model.parameters(), lr=config.cbow_lr)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # pad=0

    for epoch in range(1, config.cbow_num_epochs + 1):
        model.train()
        total_loss = 0
        for batch in dataloader:
            contexts, targets = batch
            contexts = contexts.to(config.device)
            targets = targets.to(config.device)

            logits = model(contexts)  # (B, vocab_size)
            loss = criterion(logits, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f"[CBOW Epoch {epoch}/{config.cbow_num_epochs}] Loss: {avg_loss:.4f}")





In [20]:

set_seed(Config.seed)

# 1) CSV 로딩 (train.csv에는 최소한 ["input", "output"] 컬럼이 있다고 가정)
df_train = pd.read_csv(Config.train_path)

# (입력, 출력) 쌍 생성 및 CBOW 학습용 텍스트 목록 생성
train_data = []
cbow_texts = []  # CBOW 학습을 위한 텍스트 목록 (input과 output 모두 포함)
for idx, row in df_train.iterrows():
    input_str = str(row["input"])
    output_str = str(row["output"])
    train_data.append((input_str, output_str))
    train_data.append((output_str, output_str))
    cbow_texts.append(input_str)
    cbow_texts.append(output_str)  # output 텍스트도 추가

# 2) Vocab & Tokenizer
special_tokens = [Config.PAD_TOKEN, Config.UNK_TOKEN]
vocab = build_vocab(train_data, special_tokens)
tokenizer = CharTokenizer(vocab, Config.PAD_TOKEN, Config.UNK_TOKEN)
print("Vocab size =", len(vocab))

# 3) CBOW 데이터셋 & DataLoader 생성
cbow_dataset = CBOWDataset(cbow_texts, tokenizer, Config.window_size)
cbow_loader = DataLoader(
    cbow_dataset,
    batch_size=Config.batch_size_cbow,
    shuffle=True,
    collate_fn=cbow_collate_fn
)

Vocab size = 2473


In [21]:

# # 4) CBOW 모델 초기화 및 학습
# cbow_model = CBOW(vocab_size=len(vocab), embed_dim=Config.embed_dim, pad_idx=vocab[Config.PAD_TOKEN])
# print("CBOW 모델 학습 시작...")
# train_cbow_model(cbow_model, cbow_loader, Config)
# print("CBOW 모델 학습 완료.")
# # 모델 저장 경로 설정
# cbow_model_path = "cbow_model.pth"
# torch.save(cbow_model.state_dict(), cbow_model_path)

In [22]:
# CBOW 모델 초기화 (불러오기 전에 모델 구조 정의 필요)
cbow_model_path = "cbow_model.pth"
cbow_model = CBOW(vocab_size=len(vocab), embed_dim=Config.embed_dim, pad_idx=vocab[Config.PAD_TOKEN])

# 저장된 모델 가중치 불러오기
cbow_model.load_state_dict(torch.load(cbow_model_path))

# 모델을 평가 모드로 설정
cbow_model.eval()
print(f"CBOW 모델이 '{cbow_model_path}'에서 성공적으로 불러와졌습니다.")


CBOW 모델이 'cbow_model.pth'에서 성공적으로 불러와졌습니다.


In [23]:

# 5) Token Classification 데이터셋 & DataLoader (기존)
train_dataset = TokenClassifyDataset(train_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=Config.batch_size,
    shuffle=True,
    collate_fn=token_collate_fn
)


In [24]:

# 6) 메인 모델 초기화
# model = BiMultiGRUModel(
#     vocab_size=len(vocab),
#     embed_dim=Config.embed_dim,
#     hidden_size=Config.hidden_size,
#     num_layers=Config.num_layers,
#     pad_idx=vocab[Config.PAD_TOKEN]
# )

model = BiMultiLSTMModel(
    vocab_size=len(vocab),
    embed_dim=Config.embed_dim,
    hidden_size=Config.hidden_size,
    num_layers=Config.num_layers,
    pad_idx=vocab[Config.PAD_TOKEN]
)

# model = BiMultiGRUModel(
#     vocab_size=len(vocab),
#     embed_dim=Config.embed_dim,
#     hidden_size=Config.hidden_size,
#     num_layers=Config.num_layers,
#     pad_idx=vocab[Config.PAD_TOKEN]
# )

# CBOW에서 학습한 임베딩을 메인 모델에 복사 후 freeze
model.embedding.weight.data.copy_(cbow_model.embedding.weight.data)
model.embedding.weight.requires_grad = False
print("CBOW 임베딩을 메인 모델에 적용 (freeze).")

CBOW 임베딩을 메인 모델에 적용 (freeze).


In [25]:
# 7) 메인 모델 학습
train_model(model, train_loader, Config)

[Epoch 1/20] Loss: 0.1004
[Epoch 2/20] Loss: 0.0283
[Epoch 3/20] Loss: 0.0198
[Epoch 4/20] Loss: 0.0153
[Epoch 5/20] Loss: 0.0126
[Epoch 6/20] Loss: 0.0106
[Epoch 7/20] Loss: 0.0092
[Epoch 8/20] Loss: 0.0081
[Epoch 9/20] Loss: 0.0072
[Epoch 10/20] Loss: 0.0065
[Epoch 11/20] Loss: 0.0059
[Epoch 12/20] Loss: 0.0054
[Epoch 13/20] Loss: 0.0050
[Epoch 14/20] Loss: 0.0046
[Epoch 15/20] Loss: 0.0043
[Epoch 16/20] Loss: 0.0041
[Epoch 17/20] Loss: 0.0039
[Epoch 18/20] Loss: 0.0037
[Epoch 19/20] Loss: 0.0035
[Epoch 20/20] Loss: 0.0034


In [ ]:


# 8) 추론 (여기서는 학습 데이터로 테스트하는 예시)
results = predict_model(model, train_loader, tokenizer, Config)
for inp_str, lab_str, pred_str in results:
    print(f"[입력] {inp_str} => [예측] {pred_str} (정답: {lab_str})")

# 9) 테스트 데이터 처리 및 제출 파일 생성 (필요 시)
df_test = pd.read_csv(Config.test_path)
df_test["output"] = ""
test_data = []
for idx, row in df_test.iterrows():
    input_str = str(row["input"])
    output_str = str(row["output"])
    test_data.append((input_str, output_str))
test_dataset = TokenClassifyDataset(test_data, tokenizer)
test_loader = DataLoader(
    test_dataset,
    batch_size=Config.batch_size,
    shuffle=False,
    collate_fn=token_collate_fn
)
preds = predict_model(model, test_loader, tokenizer, Config)
torch.save(model, "lstm_augmented.pth")
sub = []
for inp, lab, pred in preds:
    sub.append(pred)
df_sub = pd.read_csv('../database/submission/sample_submission.csv')
df_sub["output"] = sub
df_sub.to_csv('../database/submission/lstm_augmented_submission.csv', index=False)
for inp_str, lab_str, pred_str in preds:
    print(f"[예측] {pred_str} (정답: {lab_str})")